In [ ]:
# Import class
import os
import sys
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
class_dir = os.path.join(parent_dir, 'Scripts')
sys.path.insert(0, class_dir)
import datetime as dt
import scraper

In [ ]:
best_markets = scraper.get_best_player_markets(dt.date(2019, 1, 21))

## Show markets and order by bookie edge

In [ ]:
bookie_pr_edge_list = []
for market in best_markets:
    bookie_pr_edge_list.append(market.calc_spread())

for edge, market in sorted(zip(bookie_pr_edge_list, best_markets), key=lambda t:t[0]):
    print('Bookie edge {:.2%}'.format(edge))
    print(market, end='\n\n')

In [ ]:
for market in best_markets:
    if market.get_overs_exchange() == market.get_unders_exchange():
        print(market)